In [75]:
! python --version

Python 3.10.0


In [76]:
import json
import datetime

In [77]:
# Import original NeST JSON (CYJS)
with open('./nest.json') as f:
  nest_json = json.load(f)

In [78]:
nodes = nest_json['elements']['nodes']
edges = nest_json['elements']['edges']

In [79]:
nodeMap = {}
idMap = {}

for node in nodes:
    data = node['data']
    suid = data['SUID']
    name = data['name']
    annotation = data['Annotation']
    nodeMap[str(suid)] = {'id': name, 'name': annotation}
    idMap[name] = str(suid)


edgeMap = {}
    
for edge in edges:
    data = edge['data']
    s = nodeMap[data['source']]
    t = nodeMap[data['target']]

    source = s['id']
    target = t['id']

    entry = []
    if source in edgeMap:
        entry = edgeMap[source]
    else:
        entry = []
    
    entry.append(target)
    edgeMap[source] = entry


In [80]:
header = []
t = datetime.datetime.now()

header.append('format-version: 1.4\n')
header.append('data-version: 1.0\n')
header.append('date: ' + t.strftime("%d:%m:%Y") + ' ' + t.strftime("%X") +'\n')
header.append('property_value: http://purl.org/dc/elements/1.1/description "NeST: Nested Systems in Tumors" xsd:string\n')
header.append('property_value: http://purl.org/dc/elements/1.1/title "NeST" xsd:string\n')
header.append('property_value: IAO:0000700 NEST\n\n') # Root node

terms = edgeMap.keys()

outF = open("./fastobo_validator/nest.obo", "w")

# Add header
for l in header:
    outF.write(l)

for term in terms:
    nodeId = idMap[term]
    node = nodeMap[nodeId]
    name = node['name']
    
    line = '[Term]\n'
    outF.write(line)
    line = 'id: ' + term + '\n'
    outF.write(line)
    line = 'name: ' + name + '\n'
    outF.write(line)
    
    children = edgeMap[term]
    for child in children:
        childId = idMap[child]
        childNode = nodeMap[childId]
        name = childNode['name']
        line = 'relationship: part_of ' + child + ' ! ' + name + '\n'
        outF.write(line)

    outF.write("\n")

outF.close()


In [81]:
# Validate (Mac only)

! ./fastobo_validator/fastobo-validator nest.obo

     Parsing `nest.obo`
      Failed parsing `nest.obo`
         --> nest.obo:3:7
          |
        3 | date: 2021-10-21 22:16:35.496083␊
          |       ^---
          |
          = expected NaiveDate
